# Example of usage with price elasticity

This notebook presents a simple example of an AeroMAPS scenario in which both the demand and the offer are affected by the evolution of the energy prices.  
This work is published in chapter 11 of [this thesis](https://theses.fr/2025ESAE0001).

## Load and process

As usually, the user has to load the framework and generate a process.

In [ ]:
%matplotlib widget
from aeromaps import create_process

import gemseo as gm
import pandas as pd

from aeromaps.utils.functions import custom_logger_config

custom_logger_config(gm.configure_logger())

In [ ]:
process = create_process(configuration_file="data_elasticity/config_elasticity.yaml")

## Set up variables

### Aviation scenario

In [ ]:
process.parameters.price_elasticity = -0.9

In [ ]:
# Default ("business as usual" --> in our case considering frozen technology in 2019 and 100% kerozene) Air traffic evolution

# Aircraft fleet and operation evolution - Aircraft efficiency using the top-down approach

# ## Drop-in aircraft
# ### Mean annual efficiency gains by category [%]
# process.parameters.energy_per_ask_short_range_dropin_fuel_gain_reference_years = []
# process.parameters.energy_per_ask_short_range_dropin_fuel_gain_reference_years_values = [0.0000000001]
# process.parameters.energy_per_ask_medium_range_dropin_fuel_gain_reference_years = []
# process.parameters.energy_per_ask_medium_range_dropin_fuel_gain_reference_years_values = [0.0000000001]
# process.parameters.energy_per_ask_long_range_dropin_fuel_gain_reference_years = []
# process.parameters.energy_per_ask_long_range_dropin_fuel_gain_reference_years_values = [0.0000000001]


# # Aircraft fleet and operation evolution - Operations

# ## Values for setting the logistic function
# process.parameters.operations_final_gain = 0.0000000001  # [%]
# process.parameters.operations_start_year = 2025
# process.parameters.operations_duration = 25.0

# process.parameters.load_factor_end_year = 82.39931200000001  # 2019 value: 82.399312

# ## Growth rate by category [%]
# process.parameters.cagr_passenger_short_range_reference_periods = []
# process.parameters.cagr_passenger_short_range_reference_periods_values = [3.0]
# process.parameters.cagr_passenger_medium_range_reference_periods = []
# process.parameters.cagr_passenger_medium_range_reference_periods_values = [3.0]
# process.parameters.cagr_passenger_long_range_reference_periods = []
# process.parameters.cagr_passenger_long_range_reference_periods_values = [3.0]
# process.parameters.cagr_freight_reference_periods = []
# process.parameters.cagr_freight_reference_periods_values = [3.0]

In [ ]:
# Aircraft energy - Introduction of alternative drop-in fuels --> none here


## Share of alternative fuels in the drop-in fuel mix (the rest being supplemented by kerosene) [%]
process.parameters.biofuel_share_reference_years = [2020, 2030, 2040, 2050]
process.parameters.biofuel_share_reference_years_values = [0.0, 0.0, 0.0, 0.0]
process.parameters.electrofuel_share_reference_years = [2020, 2030, 2040, 2050]
process.parameters.electrofuel_share_reference_years_values = [0.0, 0.0, 0.0, 0.0]

In [ ]:
# Cost parameters

## Carbon tax
process.parameters.carbon_tax_reference_years = [2020, 2030, 2040, 2050]
process.parameters.carbon_tax_reference_years_values = [
    500,
    500,
    500,
    500,
]  # 2020 value differs from the value of Quinet (54 €/tCO2)

In [ ]:
# With the eleasticity there is a coupling llop bewteen airfare and traffic.
# The solver needs to enter in the loop at some point, and thus some coupling varaibles needs to have a default value. If not done in the models, one can do it here or in the input files.

process.parameters.airfare_per_rpk = pd.Series(
    0.082949239,
    index=range(process.parameters.historic_start_year, process.parameters.end_year + 1),
)

In [ ]:
process.compute()

In [ ]:
process.mda_chain.plot_residual_history(show=True, save=False, fig_size=(10.0, 4.0))

In [ ]:
rpk_with_elast = process.data["vector_outputs"]["rpk"]
rpk_no_elast = process.data["vector_outputs"]["rpk_no_elasticity"]

In [ ]:
rpk_with_elast / rpk_no_elast

In [ ]:
process.data["vector_outputs"]["total_cost_per_rpk_without_extra_tax"][2019]

In [ ]:
process.data["vector_outputs"]["total_cost_per_rpk_without_extra_tax"][2019] + 0.003983

In [ ]:
process.plot("air_transport_co2_emissions")

In [ ]:
process.plot("airfare_breakdown")

In [ ]:
from aeromaps.utils.functions import clean_notebooks_on_tests

clean_notebooks_on_tests(globals())